In [27]:
import pandas as pd
import sqlite3
import time
import requests
import concurrent.futures
from bs4 import BeautifulSoup
import warnings
import math
warnings.filterwarnings('ignore')

In [28]:
#Retrieving Names of Categories
types_df = pd.read_csv("BananaRepublic listOfTypes.csv")
categories = types_df['Type'].to_list()
categories

['men-clothing',
 'women-clothing',
 'footwear',
 'watches',
 'wallets',
 'bags',
 'jewellery',
 'belts',
 'ties',
 'cufflinks',
 'pocket-squares',
 'caps',
 'hats',
 'scarves',
 'gloves',
 'phones-cases',
 'rings',
 'wristwear',
 'socks',
 'bracelets',
 'chains']

In [29]:
# Gender Seperation
men_categories = ['men-clothing']
men_categories.extend(["men-" + cat for cat in categories[2:]])
women_categories = ['women-clothing']
women_categories.extend(["women-" + cat for cat in categories[2:]])

In [30]:
base_url = "https://bananarepublic.gap.com"
headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0"}
per_page_total = 200
main_url = "https://core.dxpapi.com/api/v1/core/?account_id=6105&auth_key=&domain_key=bananarepublic&request_id=2803065648391&_br_uid_2=uid%3D9716267812900%3Av%3D12.0%3Ats%3D1630514037116%3Ahc%3D20&url=https://bananarepublic.gap.com/index.html%23brm-search%3Frequest_type=search&search_type=keyword&q={}&l={}&br_origin=searchbox&realm=prod&ref_url=https://bananarepublic.gap.com/&request_type=search&rows=200&start={}&facet.limit=300&fl=sku_color,style_color_id,sku_sizes,swatch_image_attribute,pid,title,brand,price,maxSalePrice,minSalePrice,sale_price,price_type,promotions,thumb_image,sku_thumb_images,sku_swatch_images,sku_color_group,url,price_range,sale_price_range,description,is_live,score,defaultColorMarketingMessage,styleMarketingMessage&stats.field=sale_price&segment=customer_tier:160481&"

In [31]:
#Number of pages
def get_page_nums(categories):
    
    nums_lst = []
    
    for category in categories:
        
        res = requests.session().get(main_url.format(category, category, "0")).json()
        pages_total = math.ceil(res['response']['numFound']/per_page_total)
        print(category, " :- ", pages_total)
        nums_lst.append(pages_total)
    
    return nums_lst

print("Per Page Total products = \n", per_page_total)
page_nums_men = get_page_nums(men_categories)
print("\n")
page_nums_women = get_page_nums(women_categories)

Per Page Total products = 
 200
men-clothing  :-  4
men-footwear  :-  1
men-watches  :-  0
men-wallets  :-  1
men-bags  :-  1
men-jewellery  :-  0
men-belts  :-  1
men-ties  :-  1
men-cufflinks  :-  1
men-pocket-squares  :-  1
men-caps  :-  1
men-hats  :-  1
men-scarves  :-  1
men-gloves  :-  1
men-phones-cases  :-  0
men-rings  :-  1
men-wristwear  :-  0
men-socks  :-  1
men-bracelets  :-  0
men-chains  :-  0


women-clothing  :-  8
women-footwear  :-  1
women-watches  :-  1
women-wallets  :-  1
women-bags  :-  1
women-jewellery  :-  0
women-belts  :-  2
women-ties  :-  1
women-cufflinks  :-  0
women-pocket-squares  :-  1
women-caps  :-  1
women-hats  :-  1
women-scarves  :-  1
women-gloves  :-  1
women-phones-cases  :-  1
women-rings  :-  1
women-wristwear  :-  0
women-socks  :-  1
women-bracelets  :-  1
women-chains  :-  1


In [36]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [37]:
def extract(item, cat, gender):
    
    try:
        website = base_url

        # product link
        product_link = base_url + item['url']

        # product name
        product_name = item['title']

        # product brand
        product_brand = "Banana Republic"
        if item['brand'] != "":
            product_brand = item['brand']

        # product category
        product_category = cat

        # sizes available
        sizes = []
        
        for size in item['variants'][0]['sku_sizes']:
            sizes.append(size)

        sizes = ",".join(sizes)
        
        
        # price
        price = item['minSalePrice']

        # mrp
        mrp = price

        # gender
        gender = gender
        
        
        # description
        temp_description = item['description'].replace('\\u', ' ').replace('\\', '').replace('002c', '')
        find_key = "."
        index = temp_description.find(find_key)
        description = temp_description[2:index]
        
        
        # primary image link
        p_img_link = "https://bananarepublic.gap.com/"+item['thumb_image']

        # secondary image links
        sec_links = []
        for elem in item['variants']:
            for img in elem['sku_thumb_images']:
                sec_links.append(base_url + img)

        sec_links = ",".join(sec_links)

    except:
#        print('here')
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(description)
        Gender.append(gender)
        Cat_list.append(product_category)
        MRP.append(mrp)
        Price.append(price)
        Primary_Image_Link.append(p_img_link)
        Secondary_Image_Link.append(sec_links)
        Size_Available.append(sizes)
               

In [38]:
"""
Main Function 
"""


"""
Men's Section
"""

data = []

counter = 0
start = time.time()

for category in range(len(men_categories)):
    
    for page in range(page_nums_men[category]):
        
        page_data = requests.session().get(main_url.format(men_categories[category], men_categories[category], str(page*per_page_total))).json()
        
        for product in page_data['response']['docs']:
            extract(product, men_categories[category], "men")
        
        counter += len(data)
        
        print("Total Amount scraped :- ", counter, sep="  ")
        print("Current Category:- ", men_categories[category])
        print("\nTime Elapsed:- ", round((time.time() - start)/60, 2), "mins\n")
        print()
        
        data = []
    

end = time.time()
print("\nTotal Time Elapsed:- ", round((time.time() - start)/60, 2), "mins\n\n")


"""
Women's Section

"""


data = []

counter = 0
start = time.time()

for category in range(len(women_categories)):
    
    for page in range(page_nums_women[category]):
        
        page_data = requests.session().get(main_url.format(women_categories[category], women_categories[category], str(page*per_page_total))).json()
        
        for product in page_data['response']['docs']:
            extract(product, women_categories[category], "women")
        
        counter += len(data)
        
        
        print("Total Amount scraped :- ", counter, sep="  ")
        print("Current Category:- ", women_categories[category])
        print("\nTime Elapsed:- ", round((time.time() - start)/60, 2), "mins\n")
        print()
        
        data = []
    

end = time.time()
print("\nTotal Time Elapsed:- ", round((time.time() - start)/60, 2), "mins\n\n")

Total Amount scraped :-   0
Current Category:-  men-clothing

Time Elapsed:-  0.03 mins


Total Amount scraped :-   0
Current Category:-  men-clothing

Time Elapsed:-  0.07 mins


Total Amount scraped :-   0
Current Category:-  men-clothing

Time Elapsed:-  0.1 mins


Total Amount scraped :-   0
Current Category:-  men-clothing

Time Elapsed:-  0.13 mins


Total Amount scraped :-   0
Current Category:-  men-footwear

Time Elapsed:-  0.15 mins


Total Amount scraped :-   0
Current Category:-  men-wallets

Time Elapsed:-  0.18 mins


Total Amount scraped :-   0
Current Category:-  men-bags

Time Elapsed:-  0.21 mins


Total Amount scraped :-   0
Current Category:-  men-belts

Time Elapsed:-  0.25 mins


Total Amount scraped :-   0
Current Category:-  men-ties

Time Elapsed:-  0.28 mins


Total Amount scraped :-   0
Current Category:-  men-cufflinks

Time Elapsed:-  0.3 mins


Total Amount scraped :-   0
Current Category:-  men-pocket-squares

Time Elapsed:-  0.36 mins


Total Amount scra

In [39]:
out=pd.DataFrame({
    
    "Website":'https://bananarepublic.gap.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [40]:
out.shape

(3015, 13)

In [44]:
con=sqlite3.connect("banana__republic.db")  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(Product_Link)):
    try:
        cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://bananarepublic.gap.com/',Product_Link[i],Product_Name[i],Product_Brand[i],Cat_list[i],Size_Available[i],Price[i],MRP[i],Gender[i],Description[i],Primary_Image_Link[i],Secondary_Image_Link[i],''))
    except:
        pass
con.commit()
con.close()